# Install Requirements

Import Huggingface modules

In [10]:
# Install necessary libraries
!pip install transformers datasets evaluate accelerate scikit-learn pandas

In [11]:
%%writefile requirements.txt
transformers
datasets
evaluate
accelerate

Overwriting requirements.txt


In [12]:
! pip install -r requirements.txt

In [13]:
# Import libraries
import pandas as pd
from sklearn.model_selection import train_test_split
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from sklearn.metrics import accuracy_score, f1_score
import torch

In [14]:
# Check if GPU is available (optional but recommended for faster training)
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

Using device: cpu


## Data Load and Preprocess

In [15]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful

In [ ]:
# Load the dataset
train = pd.read_csv("/content/drive/MyDrive/Text_Mining/train.csv")
test = pd.read_csv("/content/drive/MyDrive/Text_Mining/test.csv")

In [ ]:
train.head()

Class Index                                              Title  \
0            3  Wall St. Bears Claw Back Into the Black (Reuters)   
1            3  Carlyle Looks Toward Commercial Aerospace (Reu...   
2            3    Oil and Economy Cloud Stocks' Outlook (Reuters)   
3            3  Iraq Halts Oil Exports from Main Southern Pipe...   
4            3  Oil prices soar to all-time record, posing new...   

                                         Description  
0  Reuters - Short-sellers, Wall Street's dwindli...  
1  Reuters - Private investment firm Carlyle Grou...  
2  Reuters - Soaring crude prices plus worries\ab...  
3  Reuters - Authorities have halted oil export\f...  
4  AFP - Tearaway world oil prices, toppling reco...

# Pre-Processing

Rename Column Class Index

In [ ]:
# Rename the 'Class Index' class for consistency
train.rename(columns={'Class Index': 'labels'}, inplace=True)
test.rename(columns={'Class Index': 'labels'}, inplace=True)

# ----> Check and adjust the labels to be in the range 0-3 <----
train['labels'] = train['labels'] - 1  # Assuming your labels start from 1
test['labels'] = test['labels'] - 1  # Assuming your labels start from 1
train.head()

labels                                              Title  \
0       2  Wall St. Bears Claw Back Into the Black (Reuters)   
1       2  Carlyle Looks Toward Commercial Aerospace (Reu...   
2       2    Oil and Economy Cloud Stocks' Outlook (Reuters)   
3       2  Iraq Halts Oil Exports from Main Southern Pipe...   
4       2  Oil prices soar to all-time record, posing new...   

                                         Description  
0  Reuters - Short-sellers, Wall Street's dwindli...  
1  Reuters - Private investment firm Carlyle Grou...  
2  Reuters - Soaring crude prices plus worries\ab...  
3  Reuters - Authorities have halted oil export\f...  
4  AFP - Tearaway world oil prices, toppling reco...

Concatenate Column Title and Description into 'text'

In [ ]:
# concatinating the 'title' and 'description' column as 'text'
train['text']=(train['Title']+train['Description'])
train.drop(columns=['Title','Description'],axis=1,inplace=True)
train.head()

# concatinating the 'title' and 'description' column as 'text'
test['text']=(test['Title']+test['Description'])
test.drop(columns=['Title','Description'],axis=1,inplace=True)
test.head()

labels                                               text
0       2  Fears for T N pension after talksUnions repres...
1       3  The Race is On: Second Private Team Sets Launc...
2       3  Ky. Company Wins Grant to Study Peptides (AP)A...
3       3  Prediction Unit Helps Forecast Wildfires (AP)A...
4       3  Calif. Aims to Limit Farm-Related Smog (AP)AP ...

Check for Missing values

In [ ]:
train.isnull().sum()

labels    0
text      0
dtype: int64

In [ ]:
test.isnull().sum()

labels    0
text      0
dtype: int64

Check for duplicated rows

In [ ]:
train.duplicated().sum()

0

In [ ]:
test.duplicated().sum()

0

Display number of rows and columns

In [ ]:
train_shape = train.shape
test_shape = test.shape

print(f"Train Number of rows and columns: {train_shape}")
print(f"Test Number of rows and columns: {test_shape}")

Train Number of rows and columns: (120000, 2)
Test Number of rows and columns: (7600, 2)


Check if the classes are balanced

In [ ]:
# Assuming 'labels' is the target variable column
train_class_distribution = train['labels'].value_counts(normalize=True)
test_class_distribution = test['labels'].value_counts(normalize=True)

print("Training Set Class Distribution:")
print(train_class_distribution)

print("\nTest Set Class Distribution:")
print(test_class_distribution)

Training Set Class Distribution:
labels
2    0.25
3    0.25
1    0.25
0    0.25
Name: proportion, dtype: float64

Test Set Class Distribution:
labels
2    0.25
3    0.25
1    0.25
0    0.25
Name: proportion, dtype: float64


## Split the training set

In [ ]:
# Step 2: Split the Training Data into Training and Validation Sets
train_data, val_data = train_test_split(train, test_size=0.3, random_state=42)

# Tokenization

In [ ]:
# Tokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

# Tokenization function
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=512)

# Step 4: Tokenize the Datasets
# Convert Pandas DataFrames to Hugging Face Dataset format
train_dataset = Dataset.from_pandas(train_data)
val_dataset = Dataset.from_pandas(val_data)
test_dataset = Dataset.from_pandas(test)

# Apply tokenization
train_dataset = train_dataset.map(tokenize_function, batched=True)
val_dataset = val_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

# Set format for PyTorch tensors
train_dataset = train_dataset.with_format("torch")
val_dataset = val_dataset.with_format("torch")
test_dataset = test_dataset.with_format("torch")

Map:   0%|          | 0/84000 [00:00<?, ? examples/s]

Map:   0%|          | 0/36000 [00:00<?, ? examples/s]

Map:   0%|          | 0/7600 [00:00<?, ? examples/s]

# Model

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=4)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Step 6: Define Evaluation Metrics
def compute_metrics(pred):
    predictions, labels = pred
    predictions = predictions.argmax(axis=1)
    acc = accuracy_score(labels, predictions)
    f1 = f1_score(labels, predictions, average="weighted")
    return {"accuracy": acc, "f1": f1}

In [ ]:
# Step 7: Training Arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=3e-5,
    optim='adamw_torch',
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    report_to="none"
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of ðŸ¤— Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
# Step 8: Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

In [ ]:
# Step 9: Train the Model
trainer.train()

TrainOutput(global_step=15750, training_loss=0.15550763762848718, metrics={'train_runtime': 11707.1699, 'train_samples_per_second': 21.525, 'train_steps_per_second': 1.345, 'total_flos': 6.6305176584192e+16, 'train_loss': 0.15550763762848718, 'epoch': 3.0})

### Save the Model

In [ ]:
# Save the model and tokenizer
trainer.save_model("./fine_tuned_bert")  # Saves the model
tokenizer.save_pretrained("./fine_tuned_bert")  # Saves the tokenizer
print("Model and tokenizer saved successfully.")


Model and tokenizer saved successfully.


# Visualization

In [ ]:
import matplotlib.pyplot as plt

# Example: Plot training loss
training_loss = trainer.state.log_history
# Filter out entries that don't contain the 'loss' key
loss_entries = [entry for entry in training_loss if 'loss' in entry]
epochs = range(1, len(loss_entries) + 1) # adjust epochs range to training loss values

plt.plot(epochs, [entry['loss'] for entry in loss_entries], label="Training Loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.title("Training Loss over Epochs")
plt.legend()
plt.show()

In [ ]:
import matplotlib.pyplot as plt

# Example: Plot validation loss
validation_loss = trainer.state.log_history
# Filter out entries that don't contain the 'loss' key
loss_entries = [entry for entry in validation_loss if 'loss' in entry]
epochs = range(1, len(loss_entries) + 1) # adjust epochs range to training loss values

plt.plot(epochs, [entry['loss'] for entry in loss_entries], label="Validation Loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.title("Validating Loss over Epochs")
plt.legend()
plt.show()

# Evaluation

In [ ]:
# Step 10: Evaluate on the Test Set
test_results = trainer.evaluate(test_dataset)

print("Final Test Set Performance:")
print(f"Accuracy: {test_results['eval_accuracy']:.4f}")
print(f"F1 Score: {test_results['eval_f1']:.4f}")

Final Test Set Performance:
Accuracy: 0.9454
F1 Score: 0.9455


In [ ]:
from sklearn.metrics import classification_report

# Get predictions on the test dataset
predictions = trainer.predict(test_dataset)

# Convert predictions to class labels
predicted_labels = predictions.predictions.argmax(axis=1)

# True labels
true_labels = predictions.label_ids

# Generate the classification report
report = classification_report(true_labels, predicted_labels, target_names=['World', 'Sports', 'Business', 'Sci/Tech'])
print("Classification Report:")
print(report)


Classification Report:
              precision    recall  f1-score   support

       World       0.97      0.95      0.96      1900
      Sports       0.99      0.99      0.99      1900
    Business       0.92      0.91      0.91      1900
    Sci/Tech       0.91      0.93      0.92      1900

    accuracy                           0.95      7600
   macro avg       0.95      0.95      0.95      7600
weighted avg       0.95      0.95      0.95      7600

